# <u>T16 HOI HUB</u>

## Import Libraries

In [ ]:
from ipywidgets import widgets
from IPython.display import display
import os

## Data Exploration

In [ ]:
video_folder = os.listdir('data/video')

list_of_video = {}
for video in video_folder:
    # remove hidden file especially for macOS
    if not video.startswith('.'):
        list_of_video[video] = video

In [ ]:
video_dropdown = widgets.Dropdown(
    options = list_of_video,
    description = 'Videos',
)

# video dropdown onchange function
def video_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']

# display video dropdown
video_dropdown.observe(video_on_change)
display(video_dropdown)

## Inferenece

## Training HOI ML Model

## Evaluate